In [1]:
from __future__ import division

import PyDSTool as dst
import numpy as np

from functools import wraps
import inspect

#Coupled oscillators example

![coupled oscillators](http://tutorial.math.lamar.edu/Classes/DE/SystemsModeling_files/image002.gif)

The dynamical system corresponding to the pictured scenario can be derived by considering the free body diagrams of the masses. We will not undertake that procedure here, but instead refer the reader to standard resources on classical physics. Suffice it to say that the dynamical system in question boils down to:

\begin{align}
    m_1\ddot{x}_1 &= -k_1x_1 + k_2(x_2 - x_1)  \\
    m_2\ddot{x}_2 &= -k_3x_2 - k_2(x_2 - x_1) 
\end{align}

We can rewrite this system as a system of first order ODEs:

\begin{align}
    \dot{x}_1 &= v_1 \\
    m_1\dot{v}_1 &= -(k_1 + k_2)x_1 + k_2x_2 \\
    \dot{x}_2 &= v_2 \\
    m_2\ddot{v}_2 &= -(k_2 + k_3)x_2 + k_2x_1
\end{align}

Usually, it is a good idea to non-dimensionalize the system and work with a reduced set of parameters, but for the sake of an example where we want to illustrate the use of objects from the `ModelSpec` hierarchy, we will leave the system as is. Let us define which generators our `ModelSpec` will be compatible with, and create basic `Component` and `LeafComponent` classes.

In [2]:
compatibleGenerators = dst.findGenSubClasses('ODEsystem') # compatible with ODESystem type generators

class ODEComponent(dst.Component):
    compatibleGens = compatibleGenerators
    targetLangs = dst.targetLangs  # (from common.py) -- all are compatible with ODEs

At most, we can break the system down into five components of interest. Two are masses, and three are spring elements. We also have one global system, that contains springs and masses in general. <b><font color='red'>There are two important points to note when writing definitions for custom components:</font></b>

1. Do not implement `__init__` functions! `Component`s and `LeafComponent`s both inherit from the `ModelSpec` class, which implements an `__init__` function that should not be overridden!

2. When creating a class, only supply a single `name` (a string) argument to the class. This is because the `__init__` funciton defined in `ModelSpec` has the signature `def __init__(self, name):`.

In [83]:
class Spring(ODEComponent):
    pass

class Mass(ODEComponent):
    pass
    
    
class System(ODEComponent):
    pass

`Spring` and `Mass` components are at the same hierarchy level, so we let them contain each other. However, a `System` cannot be contained by `Spring`s or `Mass`es. Note that all `System`, `Spring` and `Mass` objects created from here onwards, using their respective class constructors will contain the updated `compatibleSubcomponents` and `compatibleContainers` attributes.

In [77]:
Spring.compatibleSubcomponents = (Mass,)
Spring.compatibleContainers = (Mass,)

Mass.compatibleSubcomponents = (Spring,)
Mass.compatibleContainers = (Spring,)

System.compatibleSubcomponents = (Mass, Spring)
System.compatibleContainers = ()

Let us write some simple functions to set the mass and stiffness of `Mass` and `String` objects respectively.

In [86]:
def set_mass(mass_component, mass):
    if dst.className(mass_component) == "Mass":
        mass_component.add(dst.Par(str(mass), 'm'))
    else:
        raise StandardError("Cannot set mass of a component that is not a Mass.")
        
def set_stiffness(spring_component, stiffness):
    if dst.className(spring_component) == "Spring":
        spring_component.add(dst.Par(str(stiffness), 'k'))
    else:
        raise StandardError("Cannot set stiffness of a component that is not a Spring.")

Let us create two masses, three springs, and a system.

In [116]:
num_springs = 3
num_masses = 2

ks = np.ones(num_springs)
ms = np.ones(num_masses)

springs = [Spring('spring{}'.format(i)) for i in xrange(num_springs)]
for index, spring in enumerate(springs):
    set_stiffness(spring, ks[index])
    
masses = [Mass('mass{}'.format(i)) for i in xrange(num_masses)]
for index, mass in enumerate(masses):
    set_mass(mass, ms[index])

full_system = System('global_system')
for component in springs + masses:
    full_system.add(component)

Now would be a good time to explore some of the self-introspection methods that `ModelSpec` objects have.

In [107]:
springs[0]

Component spring0
 Parameters: ( k )

In [109]:
springs[0].search('k')

['k']

In [112]:
springs[0].search('m')

[]

In [113]:
springs[0]._registry

{'k': regObject k (Par)}

In [117]:
full_system

Component global_system
 Components: ( Component spring2, Component spring0, Component spring1, Component mass1, Component mass0 )

In [114]:
full_system._registry

{'mass0.m': regObject m (Par),
 'mass1.m': regObject m (Par),
 'spring0.k': regObject k (Par),
 'spring1.k': regObject k (Par),
 'spring2.k': regObject k (Par)}

Let us inspect these regObjects. Here are all the methods and attributes of a regObject:

In [146]:
regObject = full_system._registry['mass0.m']

inspection_results = inspect.getmembers(regObject)

for inspect_result in inspection_results:
    member_name, member_value = inspect_result
    if "__" not in member_name:
        if type(member_value) == str:
            print "{}: '{}'".format(*inspect_result)
        else:
            print "{}: {}".format(*inspect_result)

getGlobalName: <bound method regObject.getGlobalName of regObject m (Par)>
namemap: {'m': 'mass0.m'}
obj: mass0.m
objClass: <class 'PyDSTool.Symbolic.Par'>
objLocalName: 'm'
objParentClass: <class '__main__.Mass'>
objParentName: 'mass0'


We could similarly examine a `ModelSpec` object:

In [156]:
inspection_results = inspect.getmembers(full_system)

for inspect_result in inspection_results:
    member_name, member_value = inspect_result
    if "__" not in member_name:
        if type(member_value) == str:
            print "{}: '{}'".format(*inspect_result)
        else:
            print "{}: {}".format(*inspect_result)
        print "\n"

_allSubcomponentTypes: [<class 'PyDSTool.Symbolic.Par'>, <class 'PyDSTool.Symbolic.Var'>, <class 'PyDSTool.Symbolic.Input'>, <class 'PyDSTool.Symbolic.Fun'>, <class 'PyDSTool.ModelSpec.Component'>, <class 'PyDSTool.ModelSpec.LeafComponent'>]


_componentNameMap: {Par k (ExpFuncSpec): 'spring2.k', Par m (ExpFuncSpec): 'mass1.m', Component mass1
 Parameters: ( m ): 'mass1', Component mass0
 Parameters: ( m ): 'mass0', Component spring2
 Parameters: ( k ): 'spring2', Component spring0
 Parameters: ( k ): 'spring0', Component spring1
 Parameters: ( k ): 'spring1'}


_componentTypeMap: {'Par': [Par k (ExpFuncSpec), Par m (ExpFuncSpec)], 'Spring': [Component spring0
 Parameters: ( k ), Component spring1
 Parameters: ( k ), Component spring2
 Parameters: ( k )], 'Component': [Component spring0
 Parameters: ( k ), Component spring1
 Parameters: ( k ), Component spring2
 Parameters: ( k ), Component mass0
 Parameters: ( m ), Component mass1
 Parameters: ( m )], 'Mass': [Component mass0
 Paramet

#Encoding displacement and velocity as functions for the eventual RHS

incomplete

In [157]:
full_system.components

{'mass0': Component mass0
  Parameters: ( m ), 'mass1': Component mass1
  Parameters: ( m ), 'spring0': Component spring0
  Parameters: ( k ), 'spring1': Component spring1
  Parameters: ( k ), 'spring2': Component spring2
  Parameters: ( k )}

In [169]:
for component in full_system.components:
    if 'mass' in component:
        mass_component = full_system.components[component]
        mass_component.add(dst.Var('x'))
        mass_component.add(dst.Var('v'))
        mass_component.add(dst.Fun('x', [x], 'dotv', specType='RHSfuncSpec'))
        mass_component.add(dst.Fun('v', [v], 'dotx', specType='RHSfuncSpec'))

ValueError: Name 'x' already exists in registry of object 'mass1'

In [165]:
full_system.components

{'mass0': Component mass0
  Variables: ( disp )
  Parameters: ( m ), 'mass1': Component mass1
  Variables: ( disp )
  Parameters: ( m ), 'spring0': Component spring0
  Parameters: ( k ), 'spring1': Component spring1
  Parameters: ( k ), 'spring2': Component spring2
  Parameters: ( k )}